# Model Authentication

In serverless GPU attached environments, you depend on block storage solutions like S3 to persist your model files. Other than the URI of the model file, no other relevant information regarding the model file is saved in the source code.

This exposes an interesting angle to potentially attack a ML system in production. In this notebook we will demonstrate a quick & simple process to authenticate models before loading them into memory.

As an example <CI_EXAMPLE>

## Model Training Example

Let's quicky train a MNIST model:

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 59us/sample - loss: 0.2967 - accuracy: 0.9143
Epoch 2/5
60000/60000 [==============================] - 3s 50us/sample - loss: 0.1411 - accuracy: 0.9585
Epoch 3/5
60000/60000 [==============================] - 3s 49us/sample - loss: 0.1067 - accuracy: 0.9676
Epoch 4/5
60000/60000 [==============================] - 3s 49us/sample - loss: 0.0875 - accuracy: 0.9726
Epoch 5/5
60000/60000 [==============================] - 3s 48us/sample - loss: 0.0732 - accuracy: 0.9774
10000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.07242215624339879, 0.9772]

Now, let's create a temporary `algo` directory and define a file checksum function using `sha256`:

In [2]:
import os
import shutil
import hashlib

def sha256_checksum(filename, block_size=65536):
    # Let's read in 64KB chunks
    sha256 = hashlib.sha256()
    with open(filename, "rb") as f:
        for block in iter(lambda: f.read(block_size), b""):
            sha256.update(block)
    return sha256.hexdigest()

local_tmp = "algo"
if os.path.isdir(local_tmp):
    shutil.rmtree(local_tmp)
os.makedirs(local_tmp)

model_save_path = "{}/model.h5".format(local_tmp)

model.save(model_save_path, save_format="h5")

Let's now get the file hash to checksum the model file later:

In [3]:
file_checksum = sha256_checksum(model_save_path)

Let's also define our Algorithmia API key, username & algoname:

In [4]:
API_KEY = "simXXXXXXXXXXXXXXXXXXXXXXXXX"
username = "username"
algoname = "mnist_classifier"

Let's now call out deployment script to deploy our model w/ authentication enabled:

In [5]:
%run -i "deploy.py" -k "$API_KEY" -u "$username" -a "$algoname" -m "model.py" -d "model_requirements.txt" -p ".my/$algoname" -c "$file_checksum"

Algorithm namepace: username/mnist_classifier
Algorithm URL: https://algorithmia.com/algorithms/username/mnist_classifier
Pushing source code upstream, uploading model file & compiling algorithm...
Algorithm Source Code is available at: https://algorithmia.com/algorithms/username/mnist_classifier/source
Testing new compiled algorithm via API endpoint...
Test complete!
Publishing and deploying algorithm...
Algorithm has been deployed!


Now let's test our newly authenticated model API endpoint:

In [6]:
import Algorithmia

client = Algorithmia.client(API_KEY)

sample_input = [[0]*28]*28

r = client.algo("{}/{}/0.1.0".format(username, algoname)).pipe(sample_input)
probs = r.result["prob"]

In [7]:
probs

[0.05036388710141182,
 0.14035539329051971,
 0.10898920148611067,
 0.04734822362661362,
 0.09845330566167831,
 0.2660750448703766,
 0.08395491540431976,
 0.125982403755188,
 0.026822060346603394,
 0.05165553838014603]

As seen above, the most confident prediction is for 5:

In [8]:
probs.index(max(probs))

5

You just successfully deployed your Tensorflow model!

Everytime your algorithm gets initialized during runtime, it'll also do an authentication check before processing anything.

You can call this API endpoint from anywhere: your mobile app, your website, and more...